In [1]:
# -*- coding: utf-8 -*-
import json
import math
import numpy as np
import pandas as pd
import geopandas as gpd
from shapely.geometry import Point

# ===== 입력 파일(앞서 만든 파일들) =====
FILES = {
    "morning":   "hotspots_period_morning_hot.json",
    "lunch":     "hotspots_period_lunch_hot.json",
    "afternoon": "hotspots_period_afternoon_hot.json",
    "evening":   "hotspots_period_evening_hot.json"
}

# 병합(같은 핵으로 간주)할 최대 거리(m)
MERGE_M = 120

CRS_WGS84 = "EPSG:4326"
CRS_KM    = "EPSG:5186"

# 색/반지름: 반복 강도 기반
def color_for_repeat(n):
    # 1회/2회/3회/4회 이상
    return {1:"#ffccbc", 2:"#ff8a65", 3:"#f4511e"}.get(min(n,4), "#b71c1c")

def radius_for_repeat(local_sum_mean, z_mean, n):
    # 반복이 많을수록 약간 가산
    base = 6 + 0.35*math.sqrt(max(local_sum_mean, 0)) + 1.4*max(0.0, z_mean)
    return float(min(42, base + 6*(max(1,n)-1)))

# ===== 1) 구간 파일들 로드 → 합치기 =====
parts = []
for key, path in FILES.items():
    try:
        df = pd.read_json(path)
        if df.empty: 
            continue
        df["period_key"] = key
        parts.append(df)
    except Exception as e:
        print(f"로드 실패/건너뜀: {path} ({e})")

if not parts:
    raise SystemExit("읽을 수 있는 _hot.json이 없습니다.")

df_all = pd.concat(parts, ignore_index=True)
# WGS84 → meter 좌표로 변환
g = gpd.GeoDataFrame(
    df_all.copy(),
    geometry=gpd.points_from_xy(df_all["lon"], df_all["lat"]),
    crs=CRS_WGS84
).to_crs(CRS_KM)

# ===== 2) 간단 그리디 클러스터링(대표점에 120m 이내면 같은 핵) =====
coords = np.array([[p.x, p.y] for p in g.geometry])
order = np.argsort(-df_all["GiZ"].values)  # Z 높은 것부터 대표로 삼음

rep_indices = []   # 각 클러스터의 대표 인덱스(행 번호)
clusters = []      # 각 클러스터에 포함된 원본 인덱스 목록

for i in order:
    pt = coords[i]
    assigned = False
    for cid, rep_i in enumerate(rep_indices):
        rep_pt = coords[rep_i]
        if np.hypot(pt[0]-rep_pt[0], pt[1]-rep_pt[1]) <= MERGE_M:
            clusters[cid].append(i)
            assigned = True
            break
    if not assigned:
        rep_indices.append(i)
        clusters.append([i])

print(f"클러스터 수(핵의 개수): {len(clusters)}")

# ===== 3) 각 핵별 요약 통계 계산 =====
rows = []
for cid, idxs in enumerate(clusters):
    sub = df_all.iloc[idxs].copy()
    # 대표 좌표: meter 좌표 평균 → WGS84로 역투영
    gsub = g.iloc[idxs]
    cx = float(gsub.geometry.x.mean())
    cy = float(gsub.geometry.y.mean())
    # 포인트 geometry 생성
    rep_geom = gpd.GeoSeries([Point(cx, cy)], crs=CRS_KM).to_crs(CRS_WGS84).iloc[0]
    rep_lon, rep_lat = float(rep_geom.x), float(rep_geom.y)

    periods = sorted(set(sub["period_key"]))
    repeat_n = len(periods)
    z_mean   = float(sub["GiZ"].mean())
    z_max    = float(sub["GiZ"].max())
    ls_mean  = float(sub["local_sum"].mean()) if "local_sum" in sub else float("nan")
    cc_mean  = float(sub["count"].mean())     # center_count를 'count'로 저장했었음

    # 시각화 속성
    color  = color_for_repeat(repeat_n)
    radius = radius_for_repeat(ls_mean if not math.isnan(ls_mean) else cc_mean, z_mean, repeat_n)

    rows.append({
        "lat": rep_lat,
        "lon": rep_lon,
        "repeat_n": repeat_n,
        "periods": periods,
        "GiZ_mean": z_mean,
        "GiZ_max": z_max,
        "local_sum_mean": ls_mean,
        "center_count_mean": cc_mean,
        "color": color,
        "radius": radius
    })

overlay = pd.DataFrame(rows)

# 분포 메타(몇 회 반복이 몇 개?)
dist = overlay["repeat_n"].value_counts().sort_index()
meta = {
    "merge_radius_m": MERGE_M,
    "clusters": int(len(overlay)),
    "repeat_distribution": {int(k): int(v) for k, v in dist.items()}
}

# ===== 4) 저장 =====
overlay.to_json("hotspots_multi_period_overlay.json", orient="records", force_ascii=False)
with open("overlay_meta.json", "w", encoding="utf-8") as f:
    json.dump(meta, f, ensure_ascii=False, indent=2)

print("저장 완료: hotspots_multi_period_overlay.json, overlay_meta.json")
print("분포:", meta)


클러스터 수(핵의 개수): 69
저장 완료: hotspots_multi_period_overlay.json, overlay_meta.json
분포: {'merge_radius_m': 120, 'clusters': 69, 'repeat_distribution': {1: 52, 2: 17}}
